In [1]:
import torchlensmaker as tlm
import torch

import json

from torchlensmaker.new_light_sources.light_sources_elements import NewObject2D


def describe(name, t):
    print(name)
    print(t.shape)
    print(t.min(), t.max())
    print()

optics = tlm.Sequential(
    NewObject2D(20, 5, (400, 660)),
    #tlm.ObjectAtInfinity(10., 10., material="air", wavelength=(400, 800)),
    tlm.Gap(10),
    tlm.RefractiveSurface(tlm.Sphere(15, 50), material="BK7"),
    tlm.Gap(50),
    tlm.Aperture(50)
)

print(optics)

data = tlm.default_input(dim=2, dtype=torch.float64, sampling={"base": 5, "object": 3, "wavelength": 3})
outputs = optics(data)

describe("P", outputs.P)

describe("base", outputs.rays_base)
describe("object", outputs.rays_object)
describe("wavel", outputs.rays_wavelength)

scene = tlm.show2d(optics, return_scene=True)
#json.dump(scene, open("testnb.json", "w"))


Sequential(
  (0): NewObject2D()
  (1): Gap(
    (mixed_dim): MixedDim(
      (module_2d): Translate2D()
      (module_3d): Translate3D()
    )
  )
  (2): RefractiveSurface(
    (collision_surface): CollisionSurface()
  )
  (3): Gap(
    (mixed_dim): MixedDim(
      (module_2d): Translate2D()
      (module_3d): Translate3D()
    )
  )
  (4): Aperture(
    (collision_surface): CollisionSurface()
  )
)
P
torch.Size([216, 2])
tensor(-12.9509) tensor(60.)

base
torch.Size([216])
tensor(-10.) tensor(10.)

object
torch.Size([216])
tensor(-2.5000) tensor(2.5000)

wavel
torch.Size([216])
tensor(400.) tensor(660.)

